In [1]:
#Shishir Khanal
#CMU-Optimal Controls from Jack Manchester
#Optimization: LQR using Riccati for Double Integrator System
# Double Inegrator: F = ma for unit mass 1D system where F => control u 

In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
Pkg.add("PyPlot")
using LinearAlgebra
using PyPlot
using ControlSystems

  Activating project at `~/Documents/Optimal_Control/Sims/LQR`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Documents/Optimal_Control/Sims/LQR/Project.toml`
  No Changes to `~/Documents/Optimal_Control/Sims/LQR/Manifest.toml`


In [ ]:
#Discrete Dynamics
h = 0.1 #time step
A = [1 h; 0 1]
B = [0.5*h*h; h]

In [ ]:
n = 2 #number of states
m = 1 #number of controls
Tfinal = 10.0 #final time #try larger values
N = Int(Tfinal/h)+1      #number of time steps
thist = Array(range(0, h*(N-1), step=h));

In [ ]:
#Initial Conditions
x0 = [1.0; 0]

In [ ]:
#Cost Weignts
Q = Array(1.0*I(2))
R = 0.1   #Array(1.0*I(1))
#If R is changed to 1, that means I am making controls more expensive so the solution will use less controls
Qn = Array(1.0*I(2))

In [ ]:
#Cost Function
function J(xhist, uhist)
    cost = 0.5*xhist[:,end]'*Qn*xhist[:,end]
    for k = 1:(N-1)
        cost = cost + 0.5*xhist[:,k]'*Q*xhist[:, k] + 0.5*(uhist[k]'*R*uhist[k])[1]
    end
    return cost
end

In [ ]:
P = zeros(n,n,N)
K = zeros(m,n,N-1)

P[:,:,N] .= Qn

#Backward Riccati recursion
#Just depends on the dynamics and the cost function
for k = (N-1):-1:1
    K[:,:,k] .= (R + B'*P[:,:,k+1]*B)\(B'*P[:,:,k+1]*A)
    P[:,:,k] .= Q + A'*P[:,:,k+1]*(A-B*K[:,:,k])
end

#Forward Rollout starting at x0
xhist = zeros(n,N)
xhist[:,1] = x0
uhist = zeros(m,N-1)
for k = 1:(N-1)
    uhist[:,k] .= -K[:,:,k]*xhist[:,k]
    xhist[:,k+1] .= A*xhist[:,k] + B*uhist[k]
end

In [ ]:
#plot x1 vs x2, u vs t
times = range(0, h*(N-1), step=h)
plot(times, xhist[1,:], label="Position")
plot(times, xhist[2,:], label="Velocity")
xlabel("Time")
legend()

In [ ]:
plot(times[1:end-1], uhist[1,:], label="control")
xlabel("Time")
legend()

In [ ]:
J(xhist, uhist)

In [ ]:
#Forward rollout with random noise
xhist = zeros(n,N)
xhist[:,1] = x0  #10.0*randn(2) #Can start anywhere
uhist = zeros(m,N-1)
for k = 1:(N-1)
    uhist[:,k] .= -K[:,:,k]*xhist[:,k]
    xhist[:,k+1] .= A*xhist[:,k] + B*uhist[k] + 0.01*randn(2) #can add noise to the dynamics
end

In [ ]:
plot(K[1,1,:],label="K1")
plot(K[1,2,:],label="K2")
xlabel("Time")
legend()

In [ ]:
#Compute infinite-horizon K matrix using ControlSystems.jl
Kinf = dlqr(A,B,Q,R)
#Compare to ours
K[:,:,1]-Kinf

In [ ]:
#Forward rollout with constant K
xhist = zeros(n,N)
xhist[:,1] = x0  #10.0*randn(2) #Can start anywhere
uhist = zeros(m,N-1)
for k = 1:(N-1)
    uhist[:,k] .= -Kinf*xhist[:,k]
    xhist[:,k+1] .= A*xhist[:,k] + B*uhist[k] #+ 0.01*randn(2)
end

In [ ]:
#Closed-loop Eigenvalues
eigvals(A-B*Kinf)